In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers

IMAGE_SIZE=256
BATCH_SIZE=32


In [3]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


Found 2152 files belonging to 3 classes.


In [3]:
class_name=dataset.class_names
class_name

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [4]:
# for image_batch, label_batch in dataset.take(1):

# test and validation split
# 80% training , 10% test 10%  


In [5]:
train_size = 0.8
len(dataset)*train_size

54.400000000000006

In [6]:
train_ds = dataset.take(54)
len(train_ds)

54

In [7]:
test_ds = dataset.skip(54)
len(test_ds)

14

In [8]:
val_size = 0.1
len(dataset)*val_size

6.800000000000001

In [9]:
val_ds = test_ds.take(6)
len(val_ds)

6

In [10]:
test_ds=test_ds.skip(6)
len(test_ds)

8

In [12]:
def get_dataset_partitions(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle = True , shuffle_size=1000):
    ds_size = len(ds)
    if shuffle:
        ds.shuffle(shuffle_size,seed=12)
    #sizes
    train_size = int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    #datasets
    train_ds=dataset.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds , val_ds , test_ds


In [13]:
train_ds,val_ds,test_ds = get_dataset_partitions(dataset)

In [14]:
len(train_ds)

54

In [15]:
len(val_ds)

6

In [16]:
len(test_ds)

8

In [17]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Resize and rescale

In [18]:
resize_and_rescale=tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
        layers.experimental.preprocessing.Rescaling(0.1/255)
    ]
)

Data Augmentation

In [19]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [20]:
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,3)
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(3,activation='softmax')
])
model.build(input_shape=input_shape)

In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 256, 256, 3)         0         
_________________________________________________________________
sequential_1 (Sequential)    (32, 256, 256, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (32, 254, 254, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 127, 127, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 125, 125, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 62, 62, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 60, 60, 64)         

In [22]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    
)

In [23]:
model.fit(train_ds,epochs=50,batch_size=BATCH_SIZE,verbose=1,validation_data=val_ds)

Epoch 1/50
54/54 [==============================] - 164s 1s/step - loss: 0.9168 - accuracy: 0.4797 - val_loss: 0.8917 - val_accuracy: 0.6163
Epoch 2/50
54/54 [==============================] - 58s 1s/step - loss: 0.7614 - accuracy: 0.6806 - val_loss: 0.6548 - val_accuracy: 0.7147
Epoch 3/50
54/54 [==============================] - 58s 1s/step - loss: 0.5967 - accuracy: 0.7726 - val_loss: 0.4973 - val_accuracy: 0.8275
Epoch 4/50
54/54 [==============================] - 58s 1s/step - loss: 0.4904 - accuracy: 0.8137 - val_loss: 0.4943 - val_accuracy: 0.8073
Epoch 5/50
54/54 [==============================] - 58s 1s/step - loss: 0.4599 - accuracy: 0.8194 - val_loss: 0.4701 - val_accuracy: 0.8137
Epoch 6/50
54/54 [==============================] - 58s 1s/step - loss: 0.4378 - accuracy: 0.8299 - val_loss: 0.6150 - val_accuracy: 0.7795
Epoch 7/50
54/54 [==============================] - 58s 1s/step - loss: 0.3925 - accuracy: 0.8466 - val_loss: 0.4742 - val_accuracy: 0.7662
Epoch 8/50
54/54 [=

In [25]:
import os.path

In [26]:
if os.path.isfile('models/potatoModel/potatoModel.h5') is False:
    model.save('models/potatoModel/potatoModel.h5')

In [29]:
from tensorflow.keras.models import load_model
newModel=load_model('models/potatoModel/potatoModel.h5')

In [30]:
newModel

In [31]:
model